In [1]:
import os
while os.getcwd().split('\\')[-1] != 'master-llm-rag-vnlaw':
    os.chdir('..')
    print(os.getcwd())

d:\DANG NHI\repos\master-llm-rag-vnlaw


In [2]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
import regex as re
import json

targets = [
    # ('https://dulieuphapluat.vn/van-ban/dat-dai-nha-o-van-ban/page/{page}', 'data/NhaDat.json'),
    ('https://dulieuphapluat.vn/van-ban/bao-hiem-van-ban/page/{page}', 'data/BaoHiem.json'),
    # ("https://dulieuphapluat.vn/van-ban/bo-may-hanh-chinh-van-ban/page/{page}", 'data/BoMayHanhChinh.json'),
    # ("https://dulieuphapluat.vn/van-ban/co-cau-to-chuc-van-ban/page/{page}", 'data/CoCauToChuc.json'),
    # ("https://dulieuphapluat.vn/van-ban/cong-nghe-thong-tin-van-ban/page/{page}", 'data/CongNgheThongTin.json'),
    # ("https://dulieuphapluat.vn/van-ban/cong-nghiep-van-ban/page/{page}", 'data/CongNghiep.json'),
    # ("https://dulieuphapluat.vn/van-ban/dan-su-van-ban/page/{page}", 'data/DanSu.json'),
    # ("https://dulieuphapluat.vn/van-ban/dich-vu-phap-ly-van-ban/page/{page}", 'data/DichVuPhapLy.json'),
    # ("https://dulieuphapluat.vn/van-ban/doanh-nghiep-van-ban/page/{page}", 'data/DoanhNghiep.json'),
    # ("https://dulieuphapluat.vn/van-ban/giao-duc-van-ban/page/{page}", 'data/GiaoDuc.json'),
    # ("https://dulieuphapluat.vn/van-ban/giao-thong-van-tai-van-ban/page/{page}", 'data/GiaoThongVanTai.json'),
    # ("https://dulieuphapluat.vn/van-ban/lao-dong-van-ban/page/{page}", 'data/LaoDong.json'),
    # ("https://dulieuphapluat.vn/van-ban/tai-chinh-ngan-hang-van-ban/page/{page}", 'data/TaiChinhNganHang.json'),
]

# Test run

In [3]:
target = targets[0]
main_url = target[0]
save_file = target[1]

print(main_url)
print(save_file)

https://dulieuphapluat.vn/van-ban/bao-hiem-van-ban/page/{page}
data/BaoHiem.json


In [ ]:
page = 1
url = main_url.format(page=page)
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

legal_doc_items = []
for item in soup.find_all(class_="legal-doc-title"):
    legal_doc_items.append(item.find('a')['href'])

legal_doc_url = legal_doc_items[0]
print(legal_doc_url)

https://dulieuphapluat.vn/van-ban/bao-hiem-van-ban/page/1


In [6]:
data = dict()
data['url'] = legal_doc_url

response = requests.get(legal_doc_url)
soup = BeautifulSoup(response.text, 'html.parser')
legal_doc_content = soup.find(class_="legal_doc_content")

In [ ]:
tags = []
for child in legal_doc_content.children:
    if isinstance(child, Tag):
        tags.append(child)

if len(tags) <= 1:
    legal_doc_content = tags[0]

tags = []
for child in legal_doc_content.children:
    if isinstance(child, Tag):
        tags.append(child)

table_encouter = 0
for tag in tags:
    if tag.name == 'table':
        table_encouter += 1
    if table_encouter == 3:
        target_tag = tag
        break

In [ ]:
# rows = []
# for tr in target_tag.find_all('tr'):
#     cols = []
#     for td in tr.find_all(['td']):
#         text = td.get_text(strip=True)
#         cols.append(text)
#     if cols:  # Only add non-empty rows
#         rows.append(' | '.join(cols))

# for row in rows:
#     print(row)

STT | Cơ sở y tế/loại dịch vụ | Giá dịch vụ | Ghi chú
1 | 2 | 3 | 4
1 | Bệnh viện hạng đặc biệt | 42.100 | 
2 | Bệnh viện hạng I | 42.100 | 
3 | Bệnh viện hạng II | 37.500 | 
4 | Bệnh viện hạng III | 33.200 | 
5 | Bệnh viện hạng IV | 30.100 | 
6 | Trạm y tế xã | 30.100 | 
7 | Hội chẩn để xác định ca bệnh khó (chuyên gia/ca; Chỉ áp dụng đối với trường hợp mời chuyên gia đơn vị khác đến hội chẩn tại cơ sở khám, chữa bệnh). | 200.000 | 
8 | Khám cấp giấy chứng thương, giám định y khoa (không kể xét nghiệm, X-quang) | 160.000 | 
9 | Khám sức khỏe toàn diện lao động, lái xe, khám sức khỏe định kỳ (không kể xét nghiệm, X- quang) | 160.000 | 
10 | Khám sức khỏe toàn diện cho người đi xuất khẩu lao động (không kể xét nghiệm, X-quang) | 450.000 | 


In [10]:
def get_text_p_div(tag):
    return tag.get_text(strip=True, separator=' ')

def get_text_tabel_div(tag):
    rows = []
    for tr in tag.find_all('tr'):
        cols = []
        for td in tr.find_all(['td']):
            text = td.get_text(strip=True)
            cols.append(text)
        if cols:
            rows.append(' | '.join(cols))
    content = '\n'.join(rows)
    content = "Bắt đầu bảng: \n" + content + "\n Kết thúc bảng."
    return content

In [11]:
table_encouter = 0
content = []
for tag in tags:
    if tag.name == 'table':
        table_encouter += 1
        if table_encouter > 2:
            content.append(get_text_tabel_div(tag))
        else:
            content.append(tag.get_text(strip=True, separator=' '))
    else:
        content.append(tag.get_text(strip=True, separator=' '))
data['content'] = content

In [12]:
content

['HỘI ĐỒNG NHÂN DÂN TỈNH KHÁNH HÒA ------- CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc --------------- Số: 06/2024/NQ-HĐND Khánh Hòa, ngày 12 tháng 7 năm 2024',
 '',
 'NGHỊ QUYẾT',
 'VỀ VIỆC BAN HÀNH MỨC GIÁ DỊCH VỤ KHÁM BỆNH, CHỮA BỆNH KHÔNG THUỘC PHẠM VI THANH TOÁN CỦA QUỸ BẢO HIỂM Y TẾ MÀ KHÔNG PHẢI LÀ DỊCH VỤ KHÁM BỆNH, CHỮA BỆNH THEO YÊU CẦU TẠI CÁC CƠ SỞ KHÁM BỆNH, CHỮA BỆNH CỦA NHÀ NƯỚC TRÊN ĐỊA BÀN TỈNH KHÁNH HÒA',
 'HỘI ĐỒNG NHÂN DÂN TỈNH KHÁNH HÒA KHÓA VII, KỲ HỌP THỨ 14',
 'Căn cứ Luật Tổ chức chính quyền địa phương ngày 19 tháng 6 năm 2015;',
 'Căn cứ Luật sửa đổi, bổ sung một số điều của Luật Tổ chức Chính phủ và Luật tổ chức chính quyền địa phương ngày 22 tháng 11 năm 2019;',
 'Căn cứ Luật Ban hành văn bản quy phạm pháp luật ngày 22 tháng 06 năm 2015 ;',
 'Căn cứ Luật sửa đổi, bổ sung một số điều của Luật Ban hành văn bản quy phạm pháp luật ngày 18 tháng 6 năm 2020;',
 'Căn cứ Luật Khám bệnh, chữa bệnh ngày 09 tháng 01 năm 2023;',
 'Căn cứ Luật Giá ngày

In [ ]:
def get_text_tabel_div(tag):
    rows = []
    for tr in tag.find_all('tr'):
        cols = []
        for td in tr.find_all(['td']):
            text = td.get_text(strip=True)
            cols.append(text)
        if cols:
            rows.append(' | '.join(cols))
    content = '\n'.join(rows)
    content = "Bắt đầu bảng: \n" + content + "\n Kết thúc bảng."
    return content

def crawl_1_legal_doc(url):
    data = dict()
    data['url'] = url

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    name = soup.find(class_="custom-breadcrumb-1")
    name = name.get_text(strip=True)
    name = name.split("»")[-1].strip()
    data['name'] = name

    legal_doc_content = soup.find(class_="legal_doc_content")

    tags = []
    for child in legal_doc_content.children:
        if isinstance(child, Tag):
            tags.append(child)
    
    if len(tags) <= 1:
        legal_doc_content = tags[0]
    
    tags = []
    for child in legal_doc_content.children:
        if isinstance(child, Tag):
            tags.append(child)

    table_encouter = 0
    content = []
    for tag in tags:
        if tag.name == 'table':
            table_encouter += 1
            if table_encouter > 2:
                content.append(get_text_tabel_div(tag))
            else:
                content.append(tag.get_text(strip=True, separator=' '))
        else:
            content.append(tag.get_text(strip=True, separator=' '))

    content = [i for i in content if i.strip() != '']
    data['content'] = content
    return data

In [ ]:
fail_urls = []

for i in range(len(targets)):
    target = targets[i]
    main_url = target[0]
    save_file = target[1]
    print("------ PROCESS SAVE:", save_file)

    main_data = []

    for page in range(1, 30):
        print("------ PROCESS PAGE:", page)
        url = main_url.format(page=page)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        legal_doc_items = []
        for item in soup.find_all(class_="legal-doc-title"):
            legal_doc_items.append(item.find('a')['href'])

        for legal_doc_url in legal_doc_items:
            try:
                print(legal_doc_url)
                data = crawl_1_legal_doc(legal_doc_url)
                main_data.append(data)
            except:
                print("Error:", legal_doc_url)
                fail_urls.append(legal_doc_url)
                continue


    if os.path.exists(os.path.dirname(save_file)) == False:
        os.makedirs(os.path.dirname(save_file))

    with open(save_file, 'w', encoding='utf-8') as f:
        json.dump(main_data, f, ensure_ascii=False, indent=4)

------ PROCESS SAVE: data/DichVuPhapLy.json
(1, 'Quyết định 418/QĐ-UBND')
(1, 'Quyết định 187/QĐ-UBND')
(1, 'Quyết định 340/QĐ-UBND')
(1, 'Quyết định 378/QĐ-UBND')
(1, 'Quyết định 439/QĐ-UBND')
(1, 'Quyết định 223/QĐ-UBND')
(1, 'Quyết định 322/QĐ-UBND')
(1, 'Quyết định 178/QĐ-UBND')
(1, 'Quyết định 238/QĐ-UBND')
(1, 'Quyết định 380/QĐ-UBND')
(2, 'Quyết định 198/QĐ-UBND')
(2, 'Quyết định 284/QĐ-UBND')
(2, 'Quyết định 379/QĐ-UBND')
(2, 'Quyết định 819/QĐ-UBND')
(2, 'Quyết định 396/QĐ-UBND')
(2, 'Kế hoạch 81/KH-UBND')
(2, 'Kế hoạch 111/KH-UBND')
(2, 'Quyết định 291/QĐ-UBND')
(2, 'Quyết định 189/QĐ-UBND')
(2, 'Quyết định 283/QĐ-UBND')
(3, 'Kế hoạch 951/KH-UBND')
(3, 'Kế hoạch 418/KH-UBND')
(3, 'Quyết định 172/QĐ-BTP')
(3, 'Kế hoạch 1146/KH-UBND')
(3, 'Kế hoạch 302/KH-UBND')
(3, 'Quyết định 503/QĐ-UBND')
(3, 'Quyết định 298/QĐ-UBND')
(3, 'Quyết định 492/QĐ-UBND')
(3, 'Quyết định 282/QĐ-UBND')
(3, 'Quyết định 637/QĐ-UBND')
(4, 'Kế hoạch 06/KH-UBND')
(4, 'Kế hoạch 12/KH-UBND')
(4, 'Quyết định

In [14]:
fail_urls = [
    'as acavasv',
    'as acavasv',
    'as acavasv',
]

In [ ]:
with open('data/fail_urls.json', 'w', encoding='utf-8') as f:
    json.dump(fail_urls, f, ensure_ascii=False, indent=4)

In [8]:
folder = "data/CongNgheThongTin"

files = os.listdir(folder)

for file in files:
    if "&" in file:
        old_file = os.path.join(folder, file)
        new_file = os.path.join(folder, file.replace("zz&", "+"))
        print((old_file, new_file))
        # os.rename(old_file, new_file)